In [3]:
from torch.utils.data import DataLoader
from resnet import ResNet
from sklearn.model_selection import train_test_split
import numpy as np
from custom_dataset import FinalDataset
from custom_dataloader import Custom_Dataloader
import torch
from torch import nn

In [4]:
file_path= "D:\\thesis_data\\filteredData.hdf5"
dataset = FinalDataset(file_path)
indices = np.arange(len(dataset))
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)
train_loader = Custom_Dataloader(dataset, train_indices,batch_size=256, shuffle=True)
val_loader = Custom_Dataloader(dataset, val_indices,batch_size=256, shuffle=True)

dataset_dataloader = Custom_Dataloader(dataset)


In [10]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(1))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader,model,loss_fn,device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.BCELoss()
model = ResNet(12,24)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)
epochs = 10

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer, device)
    test_loop(val_loader, model, loss_fn, device)
    
print("Done!")
torch.save(model.state_dict(), "model.pth")

Using device: cuda
Epoch 1
-------------------------------
loss: 0.790712 [    0/368539]
loss: 0.227973 [25600/368539]
loss: 0.632263 [51200/368539]
